# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
vacaweather_df=pd.read_csv (r"output_data/cities.csv")
print(vacaweather_df)

     Unnamed: 0            City      Lat       Lng  Max Temp(f)  Humidity  \
0             0      Butaritari   3.0707  172.7902        81.19        77   
1             1         Rikitea -23.1203 -134.9692        72.12        74   
2             2          Castro -24.7911  -50.0119        69.30        41   
3             3            Atar  20.5169  -13.0499        98.67        30   
4             4           Kapaa  22.0752 -159.3190        82.38        73   
..          ...             ...      ...       ...          ...       ...   
573         573        Takaungu  -3.6837   39.8566        75.69        85   
574         574           Aksay  47.2600   39.8700        84.25        33   
575         575        Ust-Maya  60.4167  134.5333        60.96        83   
576         576       Vancouver  49.2497 -123.1193        86.99        59   
577         577  Kulhudhuffushi   6.6221   73.0700        76.68        92   

     Cloudiness  Wind Speed Country        Date  
0            41       15.

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
gmaps.configure(g_key)

In [8]:
locations = vacaweather_df[["Lat","Lng"]]
humidity = vacaweather_df["Humidity"]

In [9]:
locations

,Lat,Lng
0,3.0707,172.7902
1,-23.1203,-134.9692
2,-24.7911,-50.0119
3,20.5169,-13.0499
4,22.0752,-159.3190
...,...,...
573,-3.6837,39.8566
574,47.2600,39.8700
575,60.4167,134.5333
576,49.2497,-123.1193


In [10]:
fig=gmaps.figure()
# df = gmaps.datasets.vacaweather_df()
heat_layer=gmaps.heatmap_layer(
        locations, weights=humidity, 
        dissipating=False, 
        max_intensity=100, 
        point_radius=1
        )
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
perftemp_df = vacaweather_df.loc[(vacaweather_df["Max Temp(f)"]>70) & (vacaweather_df["Max Temp(f)"]<80)]
# perftemp_df.count()
perfwind_df = perftemp_df.loc[(vacaweather_df["Wind Speed"]< 10)]
# perfwind_df.count()
# For Humidity; If I keep it at perfect humidity I get very low results
perfwind_df = perftemp_df.loc[(vacaweather_df["Humidity"]< 40)]
# perfwind_df.count()
perfvacaweather_df = perfwind_df[(vacaweather_df["Cloudiness"]<25)]
perfvacaweather_df




/var/folders/v0/fgqm4drj0fv_rywhmh177t240000gp/T/ipykernel_4265/2517444999.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  perfvacaweather_df = perfwind_df[(vacaweather_df["Cloudiness"]<25)]


,Unnamed: 0,City,Lat,Lng,Max Temp(f),Humidity,Cloudiness,Wind Speed,Country,Date
126,126,Melfi,40.9965,15.6514,75.24,34,0,7.72,IT,1659292013
340,340,Guarujá,-23.9931,-46.2564,73.89,33,20,9.22,BR,1659292108
395,395,Presidencia Roque Sáenz Peña,-26.7852,-60.4388,79.59,21,12,2.08,AR,1659292132
420,420,Lucapa,-8.4192,20.7447,76.71,23,0,1.70,AO,1659292141


In [12]:
perfvacaweather_df


,Unnamed: 0,City,Lat,Lng,Max Temp(f),Humidity,Cloudiness,Wind Speed,Country,Date
126,126,Melfi,40.9965,15.6514,75.24,34,0,7.72,IT,1659292013
340,340,Guarujá,-23.9931,-46.2564,73.89,33,20,9.22,BR,1659292108
395,395,Presidencia Roque Sáenz Peña,-26.7852,-60.4388,79.59,21,12,2.08,AR,1659292132
420,420,Lucapa,-8.4192,20.7447,76.71,23,0,1.70,AO,1659292141


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
# Insert column for "Hotels"
hotel_df=perfvacaweather_df[["City","Country","Lat","Lng", "Max Temp(f)", "Humidity","Cloudiness","Wind Speed","Date"]]
hotel_df["Hotel Name"] =""
display(hotel_df)
# hotel_df = perfvacaweather_df.insert(0,"Hotel Name", [max],allow_duplicates=False)
# print(hotel_df)

,City,Country,Lat,Lng,Max Temp(f),Humidity,Cloudiness,Wind Speed,Date,Hotel Name
126,Melfi,IT,40.9965,15.6514,75.24,34,0,7.72,1659292013,
340,Guarujá,BR,-23.9931,-46.2564,73.89,33,20,9.22,1659292108,
395,Presidencia Roque Sáenz Peña,AR,-26.7852,-60.4388,79.59,21,12,2.08,1659292132,
420,Lucapa,AO,-8.4192,20.7447,76.71,23,0,1.70,1659292141,


In [16]:
# geocoordinates
target_type="hotel"
Radius = 5000

params= {
        
        "radius": Radius,
        "types": "hotel",
        "key": g_key,
}
for index, row in hotel_df.iterrows():
        lat= row["Lat"]
        lng= row["Lng"]
        params ["location"] = f"{lat},{lng}"

        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        hotels = requests.get(base_url, params=params).json()
        
        try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
            
        except (KeyError, IndexError):
            print("Missing Hotel in Area")
            print("------------------")
hotel_df

    

,City,Country,Lat,Lng,Max Temp(f),Humidity,Cloudiness,Wind Speed,Date,Hotel Name
126,Melfi,IT,40.9965,15.6514,75.24,34,0,7.72,1659292013,Melfi
340,Guarujá,BR,-23.9931,-46.2564,73.89,33,20,9.22,1659292108,Santos
395,Presidencia Roque Sáenz Peña,AR,-26.7852,-60.4388,79.59,21,12,2.08,1659292132,Presidencia Roque Sáenz Peña
420,Lucapa,AO,-8.4192,20.7447,76.71,23,0,1.70,1659292141,Lucapa


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# # Create bank symbol layer
# bank_layer = gmaps.symbol_layer(
#     locations, fill_color='rgba(0, 150, 0, 0.4)',
#     stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
#     info_box_content=[f"Bank amount: {bank}" for bank in bank_rate]
# )


# fig = gmaps.figure()
# fig.add_layer(bank_layer)

# fig

In [25]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color ="rgba(0,150,0, 0.4)",
    stroke_color="rgba(0,0,150,0.4)", scale=2,
#     info_box_content=[f"Hotel: {Hotel Name}" for hotel in hotel_df]
)

# Display figure
fig = gmaps.figure()
fig.add_layer(hotel_layer)
                      
fig

Figure(layout=FigureLayout(height='420px'))